In [8]:
import numpy as np


class CustomFitness:
    """Class for generating your own fitness function.

    Parameters
    ----------
    fitness_fn: callable
        Function for calculating fitness of a state with the signature
        :code:`fitness_fn(state, **kwargs)`.

    problem_type: string, default: 'either'
        Specifies problem type as 'discrete', 'continuous', 'tsp' or 'either'
        (denoting either discrete or continuous).

    kwargs: additional arguments
        Additional parameters to be passed to the fitness function.

    Example
    -------
    .. highlight:: python
    .. code-block:: python

        >>> import mlrose_hiive
        >>> import numpy as np
        >>> def cust_fn(state, c): return c*np.sum(state)
        >>> kwargs = {'c': 10}
        >>> fitness = mlrose_hiive.CustomFitness(cust_fn, **kwargs)
        >>> state = np.array([1, 2, 3, 4, 5])
        >>> fitness.evaluate(state)
        150
    """

    def __init__(self, fitness_fn, problem_type='continuous', **kwargs):

        if problem_type not in ['discrete', 'continuous', 'tsp', 'either']:
            raise Exception("""problem_type does not exist.""")
        self.fitness_fn = fitness_fn
        self.problem_type = problem_type
        self.kwargs = kwargs

    def evaluate(self, state):
        """Evaluate the fitness of a state vector.

        Parameters
        ----------
        state: array
            State array for evaluation.

        Returns
        -------
        fitness: float
            Value of fitness function.
        """

        fitness = self.fitness_fn(state, **self.kwargs)
        return fitness

    def get_prob_type(self):
        """ Return the problem type.

        Returns
        -------
        self.prob_type: string
            Specifies problem type as 'discrete', 'continuous', 'tsp'
            or 'either'.
        """
        return self.problem_type

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import mlrose_hiive as mlrose
from time import time


In [3]:

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# One-hot encode the target labels
y = to_categorical(y)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [4]:

# Define the neural network model
def build_model():
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))
    return model

# Define the custom fitness function
def fitness_function(weights):
    model = build_model()
    start = 0
    for layer in model.layers:
        weights_shape = layer.get_weights()[0].shape
        biases_shape = layer.get_weights()[1].shape
        layer_weights = weights[start:start + np.prod(weights_shape)].reshape(weights_shape)
        start += np.prod(weights_shape)
        layer_biases = weights[start:start + np.prod(biases_shape)].reshape(biases_shape)
        start += np.prod(biases_shape)
        layer.set_weights([layer_weights, layer_biases])
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    loss, accuracy = model.evaluate(X_train, y_train, verbose=0)
    return loss


In [6]:

# Define the problem dimensions
model = build_model()
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
total_weights = sum(np.prod(layer.get_weights()[0].shape) + np.prod(layer.get_weights()[1].shape) for layer in model.layers)


In [9]:

fitness_function = CustomFitness(fitness_function)
# Define the optimization problem
problem = mlrose.ContinuousOpt(length=total_weights, fitness_fn=fitness_function, maximize=False)


In [11]:

# Initialize results containers
results = {"RHC": {}, "SA": {}, "GA": {}}
histories = {"RHC": {}, "SA": {}, "GA": {}}
training_times = {"RHC": 0, "SA": 0, "GA": 0}

# Random Hill Climbing
start_time = time()
best_state_rhc, best_fitness_rhc, fitness_curve_rhc = mlrose.random_hill_climb(problem, max_attempts=100, max_iters=1000, curve=True, random_state=42)
training_times["RHC"] = time() - start_time
results["RHC"]["state"] = best_state_rhc
results["RHC"]["fitness"] = best_fitness_rhc
histories["RHC"]["curve"] = fitness_curve_rhc


In [12]:

# Simulated Annealing
start_time = time()
best_state_sa, best_fitness_sa, fitness_curve_sa = mlrose.simulated_annealing(problem, max_attempts=100, max_iters=1000, curve=True, random_state=42)
training_times["SA"] = time() - start_time
results["SA"]["state"] = best_state_sa
results["SA"]["fitness"] = best_fitness_sa
histories["SA"]["curve"] = fitness_curve_sa


In [13]:

# Genetic Algorithm
start_time = time()
best_state_ga, best_fitness_ga, fitness_curve_ga = mlrose.genetic_alg(problem, max_attempts=100, max_iters=1000, curve=True, random_state=42, pop_size=300, mutation_prob=0.2)
training_times["GA"] = time() - start_time
results["GA"]["state"] = best_state_ga
results["GA"]["fitness"] = best_fitness_ga
histories["GA"]["curve"] = fitness_curve_ga


In [14]:

# Evaluate the optimized models on the test set
test_accuracies = {}
for algo in ["RHC", "SA", "GA"]:
    model = build_model()
    start = 0
    for layer in model.layers:
        weights_shape = layer.get_weights()[0].shape
        biases_shape = layer.get_weights()[1].shape
        layer_weights = results[algo]["state"][start:start + np.prod(weights_shape)].reshape(weights_shape)
        start += np.prod(weights_shape)
        layer_biases = results[algo]["state"][start:start + np.prod(biases_shape)].reshape(biases_shape)
        start += np.prod(biases_shape)
        layer.set_weights([layer_weights, layer_biases])
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    test_accuracies[algo] = accuracy
    print(f"Test accuracy for {algo}: {accuracy:.2f}")


Test accuracy for RHC: 1.00
Test accuracy for SA: 0.90
Test accuracy for GA: 0.93
